<a href="https://colab.research.google.com/github/jiho-kang/DL_CNN_Study/blob/main/01_gradient_descent_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
* 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [1]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd

boston = load_boston()
boston

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

{'DESCR': ".. _boston_dataset:\n\nBoston house prices dataset\n---------------------------\n\n**Data Set Characteristics:**  \n\n    :Number of Instances: 506 \n\n    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.\n\n    :Attribute Information (in order):\n        - CRIM     per capita crime rate by town\n        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.\n        - INDUS    proportion of non-retail business acres per town\n        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n        - NOX      nitric oxides concentration (parts per 10 million)\n        - RM       average number of rooms per dwelling\n        - AGE      proportion of owner-occupied units built prior to 1940\n        - DIS      weighted distances to five Boston employment centres\n        - RAD      index of accessibility to radial highways\n        - TAX      full-value property-tax rate p

In [2]:
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/Weight_update.png)


In [3]:
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate = 0.01):
    n = len(target)
    predicted = (w1*rm) + (w2*lstat) + bias
    diff = target - predicted # 실제값 - 예측값
    bias_factors = np.ones((n,))
    
    w1_update = -(2/n) * learning_rate * np.dot(rm.T, diff) # np.dot을 함으로써 sum을 해줌
    w2_update = -(2/n) * learning_rate * np.dot(lstat.T, diff)
    bias_update = -(2/n) * learning_rate * np.dot(bias_factors.T, diff)
    
    mse_loss = np.mean(np.square(diff))
    
    return bias_update, w1_update, w2_update, mse_loss

In [4]:
# # gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수. 
# # rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨. 
# # 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.
# def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    
#     # 데이터 건수
#     N = len(target)
#     # 예측 값. 
#     predicted = w1 * rm + w2*lstat + bias
#     # 실제값과 예측값의 차이 
#     diff = target - predicted
#     # bias 를 array 기반으로 구하기 위해서 설정. 
#     bias_factors = np.ones((N,))
    
#     # weight와 bias를 얼마나 update할 것인지를 계산.  
#     w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
#     w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
#     bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
#     # Mean Squared Error값을 계산. 
#     mse_loss = np.mean(np.square(diff))
    
#     # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환. 
#     return bias_update, w1_update, w2_update, mse_loss

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용. 

In [5]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # weight/bias update 값 계산 
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            if (i == 0) or ((i+1) % 100 == 0):
                print('Epoch:', i+1,'/', iter_epochs)
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', loss)
                print()
        
    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함. 
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])
scaled_features[:5]

array([[0.57750527, 0.08967991],
       [0.5479977 , 0.2044702 ],
       [0.6943859 , 0.06346578],
       [0.65855528, 0.03338852],
       [0.68710481, 0.09933775]])

In [7]:
w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.252369] w2: [0.10914761] bias: [0.45065613] loss: 592.1469169960474

Epoch: 100 / 1000
w1: [9.62405472] w2: [2.05283682] bias: [15.47316916] loss: 76.76427130847858

Epoch: 200 / 1000
w1: [11.5022803] w2: [-0.05484598] bias: [16.4532951] loss: 66.99833740927085

Epoch: 300 / 1000
w1: [12.78038167] w2: [-2.21908904] bias: [16.52264258] loss: 60.65897877390511

Epoch: 400 / 1000
w1: [13.91682547] w2: [-4.20115117] bias: [16.5330944] loss: 55.42734570065424

Epoch: 500 / 1000
w1: [14.94980404] w2: [-6.00297075] bias: [16.53824521] loss: 51.10427049189361

Epoch: 600 / 1000
w1: [15.89029715] w2: [-7.64003964] bias: [16.54161605] loss: 47.5319284654896

Epoch: 700 / 1000
w1: [16.74679738] w2: [-9.12730107] bias: [16.54357324] loss: 44.579932870966196

Epoch: 800 / 1000
w1: [17.52693159] w2: [-10.47839254] bias: [16.54427189] loss: 42.14054294794409

Epoch: 900 / 1000
w1: [18.23762719] w2: [-11.70571404] bias: [16.54384162] loss: 40.1247

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산. 

In [8]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
predicted[:15]

array([26.29894642, 24.27001746, 28.84233748, 28.55127642, 28.24493494,
       25.69289099, 21.62613759, 19.82778642, 14.05653949, 19.94509459,
       20.10969008, 21.31811718, 20.02069898, 22.87338017, 22.69777096])

In [9]:
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.298946
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.270017
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.842337
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.551276
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.244935
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.692891
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.626138
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.827786
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.056539
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.945095


### Keras를 이용하여 보스턴 주택가격 모델 학습 및 예측
* Dense Layer를 이용하여 퍼셉트론 구현. units는 1로 설정. 

In [10]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # units 설정 = 하나의 layer에 있는 node 갯수.
    # input_shape = (feature 갯수, 차원)
    # 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, bostonDF['PRICE'].values, epochs=400)


Epoch 1/400
16/16 [==============================] - 1s 1ms/step - loss: 542.3615 - mse: 542.3615
Epoch 2/400
16/16 [==============================] - 0s 2ms/step - loss: 530.1743 - mse: 530.1743
Epoch 3/400
16/16 [==============================] - 0s 1ms/step - loss: 518.1933 - mse: 518.1933
Epoch 4/400
16/16 [==============================] - 0s 1ms/step - loss: 506.5819 - mse: 506.5819
Epoch 5/400
16/16 [==============================] - 0s 1ms/step - loss: 494.9981 - mse: 494.9981
Epoch 6/400
16/16 [==============================] - 0s 1ms/step - loss: 483.8386 - mse: 483.8386
Epoch 7/400
16/16 [==============================] - 0s 2ms/step - loss: 472.8990 - mse: 472.8990
Epoch 8/400
16/16 [==============================] - 0s 2ms/step - loss: 462.1791 - mse: 462.1791
Epoch 9/400
16/16 [==============================] - 0s 2ms/step - loss: 451.5965 - mse: 451.5965
Epoch 10/400
16/16 [==============================] - 0s 2ms/step - loss: 441.3946 - mse: 441.3946
Epoch 11/400
16/16 

### Keras로 학습된 모델을 이용하여 주택 가격 예측 수행. 

In [11]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE'] = predicted
bostonDF.head(10)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE,KERAS_PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.298946,28.106625
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.270017,25.095121
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.842337,31.375834
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.551276,31.143904
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.244935,30.480211
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.692891,27.327185
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.626138,21.423622
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.827786,18.394613
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.056539,9.953367
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.945095,18.783373


# Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 한건만 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 Batch 건수만큼 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

In [12]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

(506, 14)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## SGD 기반으로 Weight/Bias update 값 구하기

In [13]:
# rm_sgd는 이제 batch size만큼 들어올거임. 지금은 그냥 SGD니까 1개만 들어옴. 아래 변수 다 한개임
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    
    # 데이터 건수
    N = target_sgd.shape[0]
    # 예측 값. 
    predicted_sgd = w1 * rm_sgd + w2*lstat_sgd + bias
    # 실제값과 예측값의 차이 
    diff_sgd = target_sgd - predicted_sgd
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))
    
    # MSE를 계산했던 부분은 이제 밖에서 계산할거임 mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

### SGD 수행하기

In [14]:
print(bostonDF['PRICE'].values.shape)
print(np.random.choice(bostonDF['PRICE'].values.shape[0], 1))
print(np.random.choice(506, 1))

(506,)
[427]
[93]


In [15]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    print()
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택. 
        stochastic_index = np.random.choice(target.shape[0], 1) # target의 shape은 (갯수, 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # SGD 기반으로 Weight/Bias의 Update를 구함.  
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        
        # SGD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if (verbose):
            if (i == 0) or ((i+1)%200==0):
                print('Epoch:', i+1,'/', iter_epochs)
                # Loss는 전체 학습 데이터 기반으로 구해야 함. ###############중요#########
                predicted = w1 * rm + w2*lstat + bias # 전체 갯수
                diff = target - predicted # 전체 갯수
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]

Epoch: 1 / 1000
w1: [0.98946158] w2: [0.08002208] bias: [1.] loss: 523.6254285201774
Epoch: 200 / 1000
w1: [11.15345637] w2: [-0.43068693] bias: [15.63170774] loss: 67.98076314216202
Epoch: 400 / 1000
w1: [14.26621253] w2: [-4.69173892] bias: [16.80524556] loss: 54.184773352279514
Epoch: 600 / 1000
w1: [15.70529429] w2: [-8.52057717] bias: [15.76766723] loss: 47.61031347380366
Epoch: 800 / 1000
w1: [17.59735851] w2: [-11.41806446] bias: [16.54007279] loss: 40.95632134715162
Epoch: 1000 / 1000
w1: [18.59072594] w2: [-13.0437515] bias: [16.87342663] loss: 38.40285510413929
##### 최종 w1, w2, bias #######
[18.59072594] [-13.0437515] [16.87342663]


In [17]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_SGD'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_SGD
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.439906
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.394043
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.954733
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.680936
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.351467
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.840615
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.752975
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.904206
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.097085
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,20.043619


## iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [18]:
# 이번에는 batch size만큼 들어옴
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    # 데이터 건수
    N = target_batch.shape[0]
    # 예측 값. 
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이 
    diff_batch = target_batch - predicted_batch
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

In [19]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 갯수만큼 데이터를 임의로 선택. 
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함. 
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        # Batch GD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            if (i == 0) or ((i+1)%200==0):
                print('Epoch:', i+1,'/', iter_epochs)
                # Loss는 전체 학습 데이터 기반으로 구해야 함.
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [20]:
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.26838742] w2: [0.09756275] bias: [0.4582] loss: 564.054502216033
Epoch: 200 / 1000
w1: [11.2802044] w2: [0.03634203] bias: [16.29739972] loss: 67.53328185126713
Epoch: 400 / 1000
w1: [13.8687651] w2: [-4.10075609] bias: [16.56593278] loss: 55.62332721077435
Epoch: 600 / 1000
w1: [15.80641394] w2: [-7.52405831] bias: [16.60538157] loss: 47.759391848423995
Epoch: 800 / 1000
w1: [17.40553661] w2: [-10.38426187] bias: [16.57877093] loss: 42.33082469901731
Epoch: 1000 / 1000
w1: [18.6875145] w2: [-12.79261875] bias: [16.40344286] loss: 38.650815537849766
##### 최종 w1, w2, bias #######
[18.6875145] [-12.79261875] [16.40344286]


In [21]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_SGD,PREDICTED_PRICE_BATCH_RANDOM
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.439906,26.048340
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.394043,24.028449
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.954733,28.567896
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.680936,28.283078
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.351467,27.972934
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.840615,25.447954
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.752975,21.402594
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.904206,19.603360
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.097085,13.860908
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,20.043619,19.725452


### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [22]:
b = [1,2,3,4]
b[1:9]

[2, 3, 4]

In [23]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    print()
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴. 
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            # Batch GD로 구한 weight/bias의 update 적용. 
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
            if verbose:
                if (i == 0) or ((i+1)%200==0):
                    print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                    # Loss는 전체 학습 데이터 기반으로 구해야 함.
                    predicted = w1 * rm + w2*lstat + bias
                    diff = target - predicted
                    mse_loss = np.mean(np.square(diff))
                    print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
                    print()
        
    return w1, w2, bias

batch size를 키우면 빠르지만 컴퓨터 성능이 좋아야 함

In [24]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, batch_size=100, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [1.]

Epoch: 1 / 1000 batch step: 0
w1: [0.22765327] w2: [0.09261079] bias: [1.42618] loss: 523.5937495327876

Epoch: 1 / 1000 batch step: 100
w1: [0.48395592] w2: [0.20281825] bias: [1.87898758] loss: 497.8524182922854

Epoch: 1 / 1000 batch step: 200
w1: [0.84842665] w2: [0.29183783] bias: [2.43335843] loss: 466.68322395774254

Epoch: 1 / 1000 batch step: 300
w1: [1.03012045] w2: [0.37472713] bias: [2.79237108] loss: 448.0855850315973

Epoch: 1 / 1000 batch step: 400
w1: [1.14785609] w2: [0.47482372] bias: [3.03319319] loss: 435.5713366632866

Epoch: 1 / 1000 batch step: 500
w1: [1.32640005] w2: [0.53479778] bias: [3.35007373] loss: 419.5514184326998

Epoch: 200 / 1000 batch step: 0
w1: [18.47275292] w2: [-12.60110566] bias: [15.86951596] loss: 39.613743745462855

Epoch: 200 / 1000 batch step: 100
w1: [18.50990386] w2: [-12.59682541] bias: [15.92148701] loss: 39.48061817856162

Epoch: 200 / 1000 batch step: 200
w1: [18.58485658] w2: [-12.58289045] bias: [16

In [25]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_SGD,PREDICTED_PRICE_BATCH_RANDOM,PREDICTED_PRICE_BATCH
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.439906,26.048340,27.394484
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.394043,24.028449,24.236665
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.954733,28.567896,30.969979
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.680936,28.283078,30.668465
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.351467,27.972934,30.033437
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.840615,25.447954,26.542167
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.752975,21.402594,20.313115
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.904206,19.603360,17.241789
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.097085,13.860908,8.354012
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,20.043619,19.725452,17.586879


### Mini BATCH GD를 Keras로 수행
* Keras는 기본적으로 Mini Batch GD를 수행

In [26]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당. 
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=100, epochs=1000)

Epoch 1/1000
6/6 [==============================] - 1s 3ms/step - loss: 546.4967 - mse: 546.4967
Epoch 2/1000
6/6 [==============================] - 0s 4ms/step - loss: 541.8507 - mse: 541.8507
Epoch 3/1000
6/6 [==============================] - 0s 6ms/step - loss: 537.3311 - mse: 537.3311
Epoch 4/1000
6/6 [==============================] - 0s 3ms/step - loss: 532.7287 - mse: 532.7287
Epoch 5/1000
6/6 [==============================] - 0s 2ms/step - loss: 528.2014 - mse: 528.2014
Epoch 6/1000
6/6 [==============================] - 0s 2ms/step - loss: 523.6542 - mse: 523.6542
Epoch 7/1000
6/6 [==============================] - 0s 4ms/step - loss: 519.1941 - mse: 519.1942
Epoch 8/1000
6/6 [==============================] - 0s 3ms/step - loss: 514.8293 - mse: 514.8293
Epoch 9/1000
6/6 [==============================] - 0s 2ms/step - loss: 510.4187 - mse: 510.4187
Epoch 10/1000
6/6 [==============================] - 0s 4ms/step - loss: 506.0229 - mse: 506.0229
Epoch 11/1000
6/6 [==========

In [27]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_SGD,PREDICTED_PRICE_BATCH_RANDOM,PREDICTED_PRICE_BATCH,KERAS_PREDICTED_PRICE_BATCH
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.439906,26.048340,27.394484,27.751959
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.394043,24.028449,24.236665,24.930891
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.954733,28.567896,30.969979,30.904530
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.680936,28.283078,30.668465,30.651634
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.351467,27.972934,30.033437,30.067129
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.840615,25.447954,26.542167,27.000416
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.752975,21.402594,20.313115,21.446548
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.904206,19.603360,17.241789,18.673147
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.097085,13.860908,8.354012,10.743446
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,20.043619,19.725452,17.586879,18.999100
